In [ ]:
import luigi
import pandas as pd
import os


## Création du dossier pour stocker les données nettoyées

In [ ]:
# Vérifiez si le dossier n'existe pas déjà

dossier = 'CleanData'

if not os.path.exists(dossier):
    # Créez le dossier
    os.makedirs(dossier, mode=0o777)
    os.makedirs(os.path.join(dossier, "2017"), mode=0o777)
    os.makedirs(os.path.join(dossier, "2022"), mode=0o777)
    print("Dossier créé avec succès.")


In [ ]:
class ReadDataDepartement(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()


    def output(self):
        return luigi.LocalTarget("./CleanData/2022/departement.csv")
    
    def output1(self):
        return luigi.LocalTarget("./CleanData/2017/departement.csv")

    def run(self):
        
        # Géneration du fichier departement pour l'année 2022
        df_2022 = pd.read_csv(self.file_path, sep=",")
        df_2022 = df_2022.drop("Etat saisie", axis=1)
        df_partis = pd.read_csv('./RawData/Partis.csv', sep=",")
        df_new = pd.DataFrame()

        for index, row in df_2022.iterrows():
            #print(f"index : {index} \n row : {row[0:18]} \n\n")
            row1 = pd.Series(row[0:1], index=df_2022.columns[0:1])
            maxVoix = 0
            rowToBeMerge = pd.Series()

            for i in range(0, 12):
                # Ligne 2
                row2 = pd.Series(row.iloc[16 + i * 6:16 + ((i + 1) * 6)])

                nbVoix = float(row2[3].replace(',', ''))
                if nbVoix > maxVoix:
                    maxVoix = nbVoix
                    rowToBeMerge = row2
            
            df_row2 = pd.DataFrame([rowToBeMerge])
            df_row2_renamed = pd.DataFrame(df_row2.values, columns=['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp'])
            merged_row = pd.concat([row1, df_row2_renamed.squeeze()], axis=0)

            # Convertir la ligne fusionnée en DataFrame avec une seule ligne
            merged_row_df = pd.DataFrame([merged_row])
            # Ajouter la ligne fusionnée au DataFrame
            df_new = pd.concat([df_new, merged_row_df], ignore_index=True)


        df_new = pd.merge(df_new, df_partis, on=['Nom', 'Prenom'], how='inner')

        df_new.to_csv(self.output().path, index=False)


        # Géneration du fichier departement pour l'année 2017
        df_2022 = pd.read_csv(self.file_path1, sep=",")
        df_new = pd.DataFrame()

        for index, row in df_2022.iterrows():
            #print(f"index : {index} \n row : {row[0:18]} \n\n")
            row1 = pd.Series(row[0:1], index=df_2022.columns[0:1])

            maxVoix = 0
            rowToBeMerge = pd.Series()

            for i in range(0, 11):
                # Ligne 2
                row2 = pd.Series(row.iloc[16 + i * 6:16 + ((i + 1) * 6)])
                
                nbVoix = float(row2[3].replace(',', ''))
                if nbVoix > maxVoix:
                    maxVoix = nbVoix
                    rowToBeMerge = row2
            
            df_row2 = pd.DataFrame([rowToBeMerge])
            df_row2_renamed = pd.DataFrame(df_row2.values, columns=['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp'])
            merged_row = pd.concat([row1, df_row2_renamed.squeeze()], axis=0)

            # Convertir la ligne fusionnée en DataFrame avec une seule ligne
            merged_row_df = pd.DataFrame([merged_row])
            # Ajouter la ligne fusionnée au DataFrame
            df_new = pd.concat([df_new, merged_row_df], ignore_index=True)


        df_new = pd.merge(df_new, df_partis, on=['Nom', 'Prenom'], how='inner')

        df_new.to_csv(self.output1().path, index=False)

## Extraire les candidats

In [ ]:
class ExtractCandidats(luigi.Task):
    file_path = luigi.Parameter()

    def requires(self):
        return ReadDataDepartement(file_path="./RawData/departement_2022.csv", file_path1="./RawData/departement_2017.csv")
    
    def output(self):
        return luigi.LocalTarget("./CleanData/candidats.csv")

    def run(self):
                
        df = pd.read_csv(self.file_path, sep=",")
       
        df = df.loc[:, ["Prenom", "Nom"]]
        df = df.drop_duplicates()
        
        df.to_csv(self.output().path, index=False)

## Extraire les données sur la délinquance

In [ ]:
class ExtractDelinquance(luigi.Task):
    file_path = luigi.Parameter()

    def output1(self):
        return luigi.LocalTarget("./CleanData/2017/delinquance.csv")
    
    def output2(self):
        return luigi.LocalTarget("./CleanData/2022/delinquance.csv")

    def complete(self):
        # Méthode complete personnalisée
        return self.output1().exists() and self.output2().exists()
    
    def run(self):

        # Création du dataset délinquance pour l'année 2017
                
        df_annee_17 = pd.read_csv(self.file_path, sep=";")
        
        df_annee_17 = df_annee_17.query('annee == 17')

        df_annee_17 = df_annee_17.rename(columns={"classe" : "Type", "Code.département" : "Code du departement", "Code.région" : "Region"})

        grouped_df_17 = df_annee_17.groupby('Code du departement')['faits'].sum()

        grouped_df_17 = grouped_df_17.reset_index()

        grouped_df_17.to_csv(self.output1().path, index=False)

        # Création du dataset délinquance pour l'année 2012
                
        df_annee_22 = pd.read_csv(self.file_path, sep=";")
        
        df_annee_22 = df_annee_22.query('annee == 22')

        df_annee_22 = df_annee_22.rename(columns={"classe" : "Type", "Code.département" : "Code du departement", "Code.région" : "Region"})

        df_annee_22 = df_annee_22.groupby('Code du departement')['faits'].sum()

        df_annee_22 = df_annee_22.reset_index()

        df_annee_22.to_csv(self.output2().path, index=False)

In [ ]:
class ExtractRSA(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/2017/rsa.csv")
    
    def output1(self):
        return luigi.LocalTarget("./CleanData//2022/rsa.csv")

    def run(self):

        # annee 2017           
        df_annee_2017 = pd.read_csv(self.file_path, sep=",")

        df_annee_2017 = df_annee_2017.rename(columns={"N° Dep" : "Code du departement", "2017" : "RSA"})
        
        colonnes = ['Code du departement', 'RSA']

        df_annee_2017 = df_annee_2017[colonnes]

        df_annee_2017.to_csv(self.output().path, index=False)


        # annee 2022          
        df_annee_2022 = pd.read_csv(self.file_path, sep=",")

        df_annee_2022 = df_annee_2022.rename(columns={"N° Dep" : "Code du departement", "2021" : "RSA"})
        
        colonnes = ['Code du departement', 'RSA']

        df_annee_2022 = df_annee_2022[colonnes]

        df_annee_2022.to_csv(self.output1().path, index=False)

## Calculer le taux de chômage par département

In [ ]:
class ExtractNombreChomeurs(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/2017/chomeurs.csv")
    
    def output1(self):
        return luigi.LocalTarget("./CleanData/2022/chomeurs.csv")

    def run(self):      
        df_annee2017 = pd.read_csv(self.file_path, sep=";")
        df_annee2017 = df_annee2017.rename(columns={"Mois" : "Code du departement"})
        df_annee2017["Code du departement"] = df_annee2017['Code du departement'].str.split().str[1]
        df_annee2017 = df_annee2017.filter(regex='Code du departement|Janvier 2017', axis=1)
        df_annee2017 = df_annee2017.rename(columns={"Janvier 2017" : "Demandeurs"})
        df_annee2017["Demandeurs"] = df_annee2017["Demandeurs"].str.replace(' ', '').astype(float)



        df_annee2022 = pd.read_csv(self.file_path, sep=";")
        df_annee2022 = df_annee2022.rename(columns={"Mois" : "Code du departement"})
        df_annee2022["Code du departement"] = df_annee2022['Code du departement'].str.split().str[1]
        df_annee2022 = df_annee2022.filter(regex='Code du departement|Janvier 2022', axis=1)
        df_annee2022 = df_annee2022.rename(columns={"Janvier 2022" : "Demandeurs"})
        df_annee2022["Demandeurs"] = df_annee2022["Demandeurs"].str.replace(' ', '').astype(float)

        

        df_annee2017.to_csv(self.output().path, index=False)
        df_annee2022.to_csv(self.output1().path, index=False)


In [ ]:
class ExtractAideSociale(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/2017/aideSociale.csv")
    
    def output1(self):
        return luigi.LocalTarget("./CleanData/2022/aideSociale.csv")

    def run(self):      
        df_annee2017 = pd.read_csv(self.file_path, sep=",")
        df_annee2022 = pd.read_csv(self.file_path1, sep=",")

        

        df_annee2017.to_csv(self.output().path, index=False)
        df_annee2022.to_csv(self.output1().path, index=False)

## Exécuter toutes les tâches

In [ ]:
class ReadAllData(luigi.Task):
    def requires(self):
        return [ExtractCandidats(file_path="./CleanData/2022/departement.csv"), 
                ExtractDelinquance(file_path="./RawData/Donnees_delinquance.csv"),
                ExtractNombreChomeurs(file_path="./RawData/demandeurs.csv"),
                ExtractRSA(file_path="./RawData/rsa.csv"),
                ExtractAideSociale(file_path="./RawData/aide_sociale_2017.csv", file_path1="./RawData/aide_sociale_2022.csv")]
    
    def run(self):
        print("lancement")

    def output(self):
        return luigi.LocalTarget('result.txt')

In [ ]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')

dossier = "./CleanData/"
Restart = False

if Restart is True:
    # Parcourir tous les fichiers du dossier
    for fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, fichier)
        # Supprimer le fichier
        os.remove(chemin_fichier)

luigi.build([ReadAllData()], local_scheduler=False, no_lock=True)